In [26]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [10]:
# Get the csv data
# df = pd.read_csv("../data/iris.data")
df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.DataFrame({
#   "Dosage": [0.0,2.0,4.0,8.0,11.0,13.0,15.0,18.0,20.0,22.0,25.0,26.0,27.0,28.0,29.0,31.0,34.0,36.0,37.0],
#   "Effectiveness": [0.0,0.0,0.0,0.0,5.0,18.0,100.0,100.0,100.0,100.0,60.0,58.0,56.0,52.0,48.0,15.0,0.0,0.0,0.0]
# })
df  


,x,y,cluster
0,1.000070,40.9378,0
1,0.997360,41.1714,0
2,0.134799,41.8113,0
3,2.475850,41.6346,0
4,-3.058700,41.3887,0
...,...,...,...
2985,98.255900,68.0158,9
2986,96.500700,67.9212,9
2987,96.885200,68.1787,9
2988,96.834000,67.9841,9


In [11]:
# Ensure categoriical data is one-hot encoded
# columns is list of ints
def one_hot_encode(df, columns):
    encoded = df.copy()
    feature_to_cols = dict()
    for col in columns:
        # Get the unique values
        unique_vals = encoded.iloc[:, col].unique()

        # Start col
        start_col = encoded.shape[1]-1

        # Now for each unique value add a col to df
        for val in unique_vals:
            col_encoded = encoded.iloc[:, col] == val
            col_encoded = [(1.0 if label == True else 0.0) for label in col_encoded]
            encoded["is_" + encoded.columns[col] + "_" + str(val)] = col_encoded
        
        end_col = encoded.shape[1]-1
        feature_to_cols[col] = list(range(start_col, end_col))

    encoded.drop(encoded.columns[columns], axis=1, inplace=True)
    return (encoded, feature_to_cols)
to_encode = [2]
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded


,x,y,is_cluster_0,is_cluster_1,is_cluster_2,is_cluster_3,is_cluster_4,is_cluster_5,is_cluster_8,is_cluster_7,is_cluster_9
0,1.000070,40.9378,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.997360,41.1714,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.134799,41.8113,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.475850,41.6346,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-3.058700,41.3887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2985,98.255900,68.0158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2986,96.500700,67.9212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2987,96.885200,68.1787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2988,96.834000,67.9841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Split the data into train and test
def split_train_test(df, percent_train, randomize = True):
  #Randomize the rows
  if randomize:
    df = df.sample(frac=1.0)

  #Create train and test data
  df_size = df.shape[0]
  train_size = df_size * percent_train

  train_data = df.iloc[:int(train_size),:]
  test_data = df.iloc[int(train_size):,:]
  return (train_data, test_data)

In [13]:
def remove_encoded_category_data_and_label(encoded, feature_to_encoded_cols, label_col):
  cols_to_remove = []
  did_remove_label_col = False
  for k,v in feature_to_encoded_cols.items():
    if label_col in v:
      encoded.drop(encoded.columns[v], axis=1, inplace=True)
      did_remove_label_col = True
  if not did_remove_label_col:
    encoded.drop(encoded.columns[label_col], axis=1, inplace=True)
    

In [14]:
# Now setup the train/test data
# We will train it to predict y based on x and cluster
label_col = 1

train_data, test_data = split_train_test(encoded, 0.7)

# Feature Data
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Label Data
train_data_labels = train_data[train_data.columns[label_col]].values.tolist()
train_data_features

,x,is_cluster_0,is_cluster_1,is_cluster_2,is_cluster_3,is_cluster_4,is_cluster_5,is_cluster_8,is_cluster_7,is_cluster_9
938,89.64990,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2222,46.94460,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1404,90.78580,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
224,47.66050,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1091,80.13220,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2885,49.98140,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
240,48.24100,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1598,90.13930,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1641,8.10291,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
# Now train the model
datapoints_per_node = 500
trained_model = Rust_Machine_Learning.RegressionTree(train_data_features_list, train_data_labels, datapoints_per_node)
trained_model.print()

 7 0.5
   8 0.5
     0 65.88024999999999
       1 0.5
         0 10.53255
           0.7333396268599034
           3.1708684147727237
         41.529926605504585
       5 0.5
         55.037127272727275
         26.514348962655596
     73.27832283737023
   97.43380802919701


In [21]:
# Test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.classify(test_data_features_list)

In [30]:
# Graph labelled data by algorithm
result = test_data_features.copy()
result["y"] = labelled_results
fig = make_subplots(rows = 1, cols = 2)
fig.add_trace(px.scatter(result, x="x", y="y"), row=1, col=1)
fig.add_trace(px.scatter(test_data, x="x", y="y"), row=1, col=2)
fig.update_layout(title_text="Labelled (left). Original (right)")
fig.show()